In [1]:
## Importamos pandas y numpy para realizar lectura y transformaciones de los arhivos
import pandas as pd
import numpy as np


In [2]:
## Generamos dataset base para realizar las tranformaciones
df_amazon_prime = pd.read_csv('./DataSets/amazon_prime_titles-score.csv')
## Creamos nueva columna con inicial de la aplicacion de Streaming
df_amazon_prime = df_amazon_prime.assign(movie='a')
df_amazon_prime['cast'] = df_amazon_prime['cast'].str.lower()
df_disney_plus = pd.read_csv('./DataSets/disney_plus_titles-score.csv')
df_disney_plus = df_disney_plus.assign(movie='d')
df_disney_plus['cast'] = df_disney_plus['cast'].str.lower()
df_hulu = pd.read_csv('./DataSets/hulu_titles-score (2).csv')
df_hulu = df_hulu.assign(movie='h')
df_netflix = pd.read_csv('./DataSets/netflix_titles-score.csv')
df_netflix = df_netflix.assign(movie='n')
df_netflix['cast'] = df_netflix['cast'].str.lower()

In [3]:
## Creammos funcion para aplicar las transformaciones de los requerimientos correspondientes
def transform(df_amazon_prime):
    df_amazon_prime.insert(0,'id',df_amazon_prime.movie.str.cat(df_amazon_prime.show_id))
    df_amazon_prime['rating'].fillna('G', inplace=True)
    df_amazon_prime.drop(['movie'], axis=1, inplace=True)
    df_amazon_prime['type'] = df_amazon_prime['type'].str.lower()
    df_amazon_prime['title'] = df_amazon_prime['title'].str.lower()
    df_amazon_prime['director'] = df_amazon_prime['director'].str.lower()
    df_amazon_prime['country'] = df_amazon_prime['country'].str.lower()
    df_amazon_prime['date_added'] = df_amazon_prime['date_added'].str.lower()
    df_amazon_prime['rating'] = df_amazon_prime['rating'].str.lower()
    df_amazon_prime['listed_in'] = df_amazon_prime['listed_in'].str.lower()
    df_amazon_prime['description'] = df_amazon_prime['description'].str.lower()
    df_amazon_prime.insert(7,'fecha',pd.to_datetime(df_amazon_prime.date_added))
    df_amazon_prime['fecha'] = df_amazon_prime['fecha'].astype('str')
    df_amazon_prime[['duration_int','duration_type']] = df_amazon_prime.duration.str.split(' ',expand=True)
    df_amazon_prime['duration_int'] = df_amazon_prime['duration_int'].astype('float')
    df_amazon_prime.replace({'Seasons':'Season'})
    df_amazon_prime['duration_type'] = df_amazon_prime['duration_type'].str.lower()
    return df_amazon_prime

In [4]:
## realizamos las trasnformacion con nuestra funcion
df_amazon_prime = transform(df_amazon_prime)
df_disney_plus = transform(df_disney_plus)
df_hulu = transform(df_hulu)
df_netflix = transform(df_netflix)

In [5]:
## unimos todos los dataset en uno solo
all_form = pd.concat([df_amazon_prime, df_disney_plus,df_hulu,df_netflix])
## Realizamos un nuevo indice para evitar errores
all_form.reset_index(inplace=True)
## verificamos la uniom de los dataset, y las columnas correspondientes
all_form.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          22998 non-null  int64  
 1   id             22998 non-null  object 
 2   show_id        22998 non-null  object 
 3   type           22998 non-null  object 
 4   title          22998 non-null  object 
 5   director       14739 non-null  object 
 6   cast           17677 non-null  object 
 7   country        11499 non-null  object 
 8   fecha          22998 non-null  object 
 9   date_added     13444 non-null  object 
 10  release_year   22998 non-null  int64  
 11  rating         22998 non-null  object 
 12  duration       22516 non-null  object 
 13  listed_in      22998 non-null  object 
 14  description    22994 non-null  object 
 15  score          22998 non-null  int64  
 16  duration_int   22516 non-null  float64
 17  duration_type  22516 non-null  object 
dtypes: flo

In [6]:
## Importamos librerias para conectarnos con mysql
import pymysql
import sqlalchemy as db
## libreria para rutas de archivos
from pathlib import Path

In [7]:
## Realizamos la coneccion con mysql
username = 'root'
password = 'Carolina15+'
host = 'localhost'
database = 'proyecto01'
database_connection = db.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(username,password,host,database))

connection = database_connection.connect()
metadata = db.MetaData()

In [8]:
## cargamos el data frame a la base de datos
all_form.to_sql('plataform', connection)

ValueError: Table 'plataform' already exists.

In [ ]:
## traemos nuestro dataframe con los cambios listo para exportar 
df_sql = pd.read_sql_query('select * from plataform',connection)
all_form = pd.DataFrame(df_sql);
all_form.drop(['index'], axis=1, inplace=True)

In [ ]:
## Nuestro dataframe se encuentra listo para realizar la api
all_form.to_csv('./DataSets/all_form.csv', index=False)

c:\Users\crist\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
